# 외부데이터세트로 일반화 성능테스트

학습에 전혀 사용하지 않은 모집단을 제주 B초등학교에서 추가로 수집하여  
서울 A초등학교로 학습한 모델이 이 외부데이터 세트인 B초등학교에서도 성능이 잘 나오는지 테스트하였다.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as ms
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
import pandas as pd 
import os

In [2]:
os.chdir("..")
path=os.getcwd()

## 제주 B초등학교 테스트 데이터셋 불러오기

In [3]:
jeju_data=pd.read_csv(path+"\\data\\6.제주초_testdataset.csv",encoding="cp949")

###  데이터 세트 로드

In [4]:
data=jeju_data.iloc[:,[1,2,3,4,5,6,8]]
train=data.iloc[:,:-1]
train_targets=data.iloc[:,-1]

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

train_targets = encoder.fit_transform(train_targets)

### 데이터 세트 셋업

In [6]:
X_test=train.to_numpy()
y_test=train_targets
y_test=y_test.reshape(-1, 1)

In [7]:
test_np = X_test

test_targets_np=y_test.astype(int)

### 생성 method 리스트

In [8]:
Generation_list = ['smote','adasyn','copulagan','ctgan','nbsynthetic']

## 모델 불러오기

### 제안한 모델

In [ ]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

results = []


def extract_tabnet_embeddings(
    tabnet_model,
    X_tensor,
    device=None,
    batch_size=5000
    ):
        if device is None:
            device = next(tabnet_model.network.parameters()).device
        
        tabnet_model.network.eval()
        tabnet_model.network.to(device)
        X_tensor = X_tensor.to(device)
        
        dataset = TensorDataset(X_tensor)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        
        all_embeddings = []
        
        with torch.no_grad():
            for (xb,) in loader:
                embeddings, _ = tabnet_model.network.forward_masks(xb)
                        
                if isinstance(embeddings, dict):
                    if 'embeddings' in embeddings:
                        final_emb = embeddings['embeddings'][-1]
                    else:
                        final_emb = list(embeddings.values())[-1]
                else:
                    # embeddings가 리스트인 경우, 마지막 decision step의 embedding 사용
                    if isinstance(embeddings, list):
                        final_emb = embeddings[-1]
                    else:
                        final_emb = embeddings
                
                # final_emb의 shape 확인
                print("Shape of final_emb:", final_emb.shape)
                
                all_embeddings.append(final_emb.cpu().numpy())
        
        final_embeddings = np.concatenate(all_embeddings, axis=0)
        # 최종 shape 출력
        print("Final embeddings shape:", final_embeddings.shape)
        return final_embeddings

for generation in Generation_list:
    # TabNet 모델 로드
    with open(path+f'\\3.Training_XAI\\model_hist\\{generation}_proposed_tabnetEmbedd.pickle', 'rb') as f:
        tabnet_clf = pickle.load(f)
        
        
    with open(path+f'\\3.Training_XAI\\scaler_hist\\{generation}_proposed_standard_scaler.pickle', 'rb') as f:
        standard_scaler = pickle.load(f)
    
    # XGBoost 모델 로드 
    with open(path+f'\\3.Training_XAI\\model_hist\\{generation}_proposed.pickle', 'rb') as f:
        proposed_model = pickle.load(f)

    # 테스트 데이터를 텐서로 변환
    X_test_torch = torch.from_numpy(test_np.astype(np.float32))
    
    # TabNet 임베딩 추출
    test_embeddings = extract_tabnet_embeddings(
        tabnet_clf,
        X_test_torch,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    test_embeddings_scaled = standard_scaler.transform(test_embeddings)
    
    # XGBoost로 예측
    predict = proposed_model.predict(test_embeddings_scaled).astype(int)
    
    # predict=predict+1
    unique_values = np.array(list(set(predict)))
    print(f"Unique values in prediction: {unique_values}")
    
    # 메트릭 계산
    accuracy = accuracy_score(test_targets_np, predict)
    f1 = f1_score(test_targets_np, predict, average='weighted')
    precision, recall = sk(test_targets_np, predict, beta=1, average='weighted')[:2]
    
    results.append({
        'Model': f"{generation}-proposed_external",
        'Accuracy': round(accuracy, 5),
        'F1score': round(f1, 5), 
        'Precision': round(precision, 5),
        'Recall': round(recall, 5)
    })

df_proposed_results = pd.DataFrame(results)

Shape of final_emb: torch.Size([3311, 6])
Final embeddings shape: (3311, 6)
Unique values in prediction: [0 1 2]
Shape of final_emb: torch.Size([3311, 6])
Final embeddings shape: (3311, 6)
Unique values in prediction: [0 1 2]
Shape of final_emb: torch.Size([3311, 6])
Final embeddings shape: (3311, 6)
Unique values in prediction: [0 1 2]
Shape of final_emb: torch.Size([3311, 6])
Final embeddings shape: (3311, 6)
Unique values in prediction: [0 1 2]
Shape of final_emb: torch.Size([3311, 6])
Final embeddings shape: (3311, 6)
Unique values in prediction: [0 1 2]


In [10]:
unique_values = np.unique(test_targets_np)
print(f"Unique values in test_targets_np: {unique_values}")
    

Unique values in test_targets_np: [0 1 2]


In [12]:
predict

array([1, 1, 1, ..., 1, 1, 1])

In [13]:
df_proposed_results

,Model,Accuracy,F1score,Precision,Recall
0,smote-proposed_external,0.81637,0.80780,0.79984,0.81637
1,adasyn-proposed_external,0.82332,0.80389,0.78653,0.82332
2,copulagan-proposed_external,0.43612,0.53980,0.78457,0.43612
3,ctgan-proposed_external,0.58653,0.66540,0.78220,0.58653
4,nbsynthetic-proposed_external,0.84144,0.81121,0.78408,0.84144


In [14]:
# 데이터프레임을 CSV 파일로 저장 (선택사항)
df_proposed_results.to_csv('proposed_model_metrics_externaltest.csv')

## Decision Tree

In [ ]:
import pickle

In [18]:
# 빈 리스트 생성하여 결과 저장
results = []

for generation in Generation_list:
    with open(path+f'\\3.Training_XAI\\model_hist\\{generation}_DT_model.pickle', 'rb') as f:
        DT_model = pickle.load(f)
    predict=DT_model.predict(test_np).astype(int)
    # 각 메트릭 계산
    accuracy = DT_model.score(test_np, test_targets_np)
    f1 = f1_score(test_targets_np, predict, average='weighted')
    precision, recall = sk(test_targets_np, predict, beta=1, average='weighted')[:2]
    
    # 결과를 딕셔너리로 저장
    result_dict = {
        'Model': f"{generation}-Decision Tree",
        'Accuracy': round(accuracy, 5),
        'F1score': round(f1, 5),
        'Precision': round(precision, 5),
        'Recall': round(recall, 5)
    }
    
    # 리스트에 딕셔너리 추가
    results.append(result_dict)

# 데이터프레임 생성
df_DT_results = pd.DataFrame(results)

# 데이터프레임을 CSV 파일로 저장 (선택사항)
# df_results.to_csv('model_metrics.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\jupyter\\Explainable Healthcare framework\\Explainable_healthcareframework\\3.Training_XAI\\model_hist\\smote_DT_model.pickle'

In [ ]:
df_DT_results

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB_model = GaussianNB()



In [ ]:
# 빈 리스트 생성하여 결과 저장
results = []

for generation in Generation_list:
    with open(f'C:\\jupyter\\Elementary-student-Weight\\GAN_XAI\\Experiment3(생성별_모델별)\\model_history\\{generation}_NB_model.pickle', 'rb') as f:
        NB_model = pickle.load(f)
    predict=NB_model.predict(test_np).astype(int)
    # 각 메트릭 계산
    accuracy = NB_model.score(test_np, test_targets_np)
    f1 = f1_score(test_targets_np, predict, average='weighted')
    precision, recall = sk(test_targets_np, predict, beta=1, average='weighted')[:2]
    
    # 결과를 딕셔너리로 저장
    result_dict = {
        'Model': f"{generation}-Naive Bayes",
        'Accuracy': round(accuracy, 5),
        'F1score': round(f1, 5),
        'Precision': round(precision, 5),
        'Recall': round(recall, 5)
    }
    
    # 리스트에 딕셔너리 추가
    results.append(result_dict)

# 데이터프레임 생성
df_Naive_results = pd.DataFrame(results)

# 데이터프레임을 CSV 파일로 저장 (선택사항)
# df_results.to_csv('model_metrics.csv')

## LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA_model= LinearDiscriminantAnalysis()


In [ ]:
# 빈 리스트 생성하여 결과 저장
results = []

for generation in Generation_list:
    with open(f'C:\\jupyter\\Elementary-student-Weight\\GAN_XAI\\Experiment3(생성별_모델별)\\model_history\\{generation}_LDA_model.pickle', 'rb') as f:
        LDA_model = pickle.load(f)
    predict=LDA_model.predict(test_np).astype(int)
    # 각 메트릭 계산
    accuracy = LDA_model.score(test_np, test_targets_np)
    f1 = f1_score(test_targets_np, predict, average='weighted')
    precision, recall = sk(test_targets_np, predict, beta=1, average='weighted')[:2]
    
    # 결과를 딕셔너리로 저장
    result_dict = {
        'Model': f"{generation}-LDA",
        'Accuracy': round(accuracy, 5),
        'F1score': round(f1, 5),
        'Precision': round(precision, 5),
        'Recall': round(recall, 5)
    }
    
    # 리스트에 딕셔너리 추가
    results.append(result_dict)

# 데이터프레임 생성
df_LDA_results = pd.DataFrame(results)

# 데이터프레임을 CSV 파일로 저장 (선택사항)
# df_results.to_csv('model_metrics.csv')

In [ ]:
df_LDA_results

## LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 데이터 로드 및 전처리 (예시)
# X 데이터는 6개의 특징 변수, y 데이터는 라벨 (1.0, 2.0, 3.0)
# X, y 데이터는 적절히 로드되어 있어야 합니다.
# 모델 정의

Logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=20)

In [ ]:
# 빈 리스트 생성하여 결과 저장
results = []
scaler=StandardScaler()
for generation in Generation_list:
    with open(f'C:\\jupyter\\Elementary-student-Weight\\GAN_XAI\\Experiment3(생성별_모델별)\\model_history\\{generation}_Logistic_model.pickle', 'rb') as f:
        Logistic_model = pickle.load(f)
    
    with open(f'C:\\jupyter\\Elementary-student-Weight\\GAN_XAI\\Experiment3(생성별_모델별)\\model_history\\{generation}_Logistic_scaler.pickle', 'rb') as f:
        scaler = pickle.load(f)

    X_test=scaler.transform(test_np)
    predict=Logistic_model.predict(X_test).astype(int)
    # 각 메트릭 계산
    accuracy = Logistic_model.score(X_test, test_targets_np)
    f1 = f1_score(test_targets_np, predict, average='weighted')
    precision, recall = sk(test_targets_np, predict, beta=1, average='weighted')[:2]
    
    # 결과를 딕셔너리로 저장
    result_dict = {
        'Model': f"{generation}-LogisticRegression",
        'Accuracy': round(accuracy, 5),
        'F1score': round(f1, 5),
        'Precision': round(precision, 5),
        'Recall': round(recall, 5)
    }
    
    # 리스트에 딕셔너리 추가
    results.append(result_dict)

# 데이터프레임 생성
df_LogisticRegression_results = pd.DataFrame(results)

# 데이터프레임을 CSV 파일로 저장 (선택사항)
# df_results.to_csv('model_metrics.csv')

In [ ]:
df_LogisticRegression_results

## GAMs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from pygam import LogisticGAM, s, f

def train_multiclass_gam(X_test, y_test):
    # 레이블 인코딩
    le = LabelEncoder()
    y_test_encoded = le.fit_transform(y_test)
    n_classes = len(np.unique(y_test_encoded))

    return y_test_encoded, le

def predict_multiclass(gam_models, X):
    # 각 클래스에 대한 예측 확률 계산
    probs = np.column_stack([
        gam.predict_proba(X) for gam in gam_models
    ])
    
    # 가장 높은 확률을 가진 클래스 선택
    return np.argmax(probs, axis=1)

def print_model_summary(gam_models, class_names):
    print("\n모델 요약:")
    for i, gam in enumerate(gam_models):
        print(f"\n클래스 {class_names[i]}:")
        print("특성별 계수 정보:")
        for j, coef in enumerate(gam.coef_):
            print(f"특성 {j + 1} - 계수: {coef:.4f}")
        print("\n모델 summary:")
        gam.summary()

In [ ]:
# 빈 리스트 생성하여 결과 저장
results = []

for generation in Generation_list:
    with open(f'C:\\jupyter\\Elementary-student-Weight\\GAN_XAI\\Experiment3(생성별_모델별)\\model_history\\{generation}_gams_model.pickle', 'rb') as f:
        GAMs_model = pickle.load(f)
    y_test_encoded, le = train_multiclass_gam(train, train_targets)
    predict = predict_multiclass(GAMs_model, train)
    accuracy = accuracy_score(y_test_encoded, predict)
 
    # 각 메트릭 계산
    f1 = f1_score(y_test_encoded, predict, average='weighted')
    precision, recall = sk(y_test_encoded, predict, beta=1, average='weighted')[:2]
    
    # 결과를 딕셔너리로 저장
    result_dict = {
        'Model': f"{generation}-GAMs",
        'Accuracy': round(accuracy, 5),
        'F1score': round(f1, 5),
        'Precision': round(precision, 5),
        'Recall': round(recall, 5)
    }
    
    # 리스트에 딕셔너리 추가
    results.append(result_dict)

# 데이터프레임 생성
df_GAMs_results = pd.DataFrame(results)

# 데이터프레임을 CSV 파일로 저장 (선택사항)
# df_results.to_csv('model_metrics.csv')

In [ ]:
df_GAMs_results

## 결과 총합

In [ ]:
df_LDA_results

In [ ]:
df_LogisticRegression_results

In [ ]:
df_ml=pd.concat([df_DT_results,df_Naive_results,df_LDA_results,df_GAMs_results])

In [ ]:
df_ml=df_ml.reset_index(drop=True)

In [ ]:
df_ml.to_csv('C:\jupyter\Elementary-student-Weight\GAN_XAI\Experiment3(생성별_모델별)\data\(하귀일)머신 러닝 일반화 성능 정리.csv',encoding='cp949',index=False)

In [ ]:
df_ml 